In [1]:
#Imports needed
import quandl
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime

## SET QUANDL API KEY VARIABLE, AND PULL IN DATA FROM QUANDLE
Quandle hosts several different market data sets. We have pulled the sentiment data sets in this code. AAII Sentiment survey reports members weekly if they are bullish, bearish, or neutral on the stock market over the next six months (source: https://ycharts.com/indicators/reports/aaii_sentiment_survey)

We are pulling the three percentages between Q1 2018 and Q4 2019.

In [2]:
# Setting the quandl API key variable within the quandl class
quandl.ApiConfig.api_key = os.getenv("QUANDL_API_KEY")

#getting the Sentiment Data from quandl

sentiment_df = pd.DataFrame(quandl.get(
    ["AAII/AAII_SENTIMENT.1", "AAII/AAII_SENTIMENT.2", "AAII/AAII_SENTIMENT.3"],
    start_date = "2018-01-01",
    end_date = "2019-12-31")
                             )

sentiment_df.columns = ['Bullish', 'Neutral', 'Bearish']
# sentiment_df.reset_index(inplace = True)
sentiment_df.head()

,Bullish,Neutral,Bearish
Date,,,
2018-01-04,0.597531,0.246914,0.155556
2018-01-11,0.486726,0.262537,0.250737
2018-01-18,0.541133,0.244973,0.213894
2018-01-25,0.454545,0.305195,0.240260
2018-02-01,0.447712,0.264706,0.287582


In [3]:
sentiment_df.to_csv('~/Fintech/GroupProject1/Resources/Data/sentiment_data_MSR.csv')

In [4]:
sentiment_data_path = ('~/Fintech/GroupProject1/Resources/Data/sentiment_data_MSR.csv')

In [5]:
sentiment_data = pd.read_csv(sentiment_data_path)
sentiment_data.head()

,Date,Bullish,Neutral,Bearish
0,2018-01-04,0.597531,0.246914,0.155556
1,2018-01-11,0.486726,0.262537,0.250737
2,2018-01-18,0.541133,0.244973,0.213894
3,2018-01-25,0.454545,0.305195,0.240260
4,2018-02-01,0.447712,0.264706,0.287582


## Interpreting these results:

We can see that the the added results for each row equals one. So each row describes surveyed overall sentiment for that week. 

In [6]:
# Make sure data type is numeric.
sentiment_data['Bullish'] = sentiment_data['Bullish'].astype(float)
sentiment_data['Neutral'] = sentiment_data['Neutral'].astype(float)
sentiment_data['Bearish'] = sentiment_data['Bearish'].astype(float)
sentiment_data.dtypes

Date        object
Bullish    float64
Neutral    float64
Bearish    float64
dtype: object

## Pull in S&P Data
Next, we pull in SP500 data. We are going to look at returns each day of the S&P. The data was pulled off of Google Sheets' finance function. The interval is daily. First pull in the daily closing price, then use the .pct_change function in pandas to determine the return between each day. 

In [26]:
#Reading S&P500 Data
csvpath = Path("SP500.csv")
sp500_data = pd.read_csv(csvpath)
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])
sp500_data['Close'] = sp500_data['Close'].astype(float)
sp500_data.set_index('Date', drop = True, inplace = True)
sp500_data.head()

,Close
Date,
1987-07-24,309.27
1987-07-27,310.65
1987-07-28,312.33
1987-07-29,315.65
1987-07-30,318.05


In [27]:
#Calculating S&P500 Daily changes in % (1 Day)
sp500_data['%Change'] = sp500_data.pct_change(1)
sp500_data.head()
aux_df = sp500_data
aux_df.head()

,Close,%Change
Date,,
1987-07-24,309.27,NaN
1987-07-27,310.65,0.004462
1987-07-28,312.33,0.005408
1987-07-29,315.65,0.010630
1987-07-30,318.05,0.007603


-------------------------

## OK you got your data, now what?
Calculating your triggers. The sentiment data showed us the percentage breakdown of the surveyed's 6 month outlook on the market.
Max Bullish (or Bearish), Minimally Bullish (or Bearish), Average Bullish (or Bearish), or Median Bullish (or Bearish).
We are pulling the maximum possible values from each column (bullish or bearish), the minimum, and the mean. Then, we are splitting the difference between the maximum and the median values. 
Remember, this is over the ENTIRE data set.

In [28]:
#Calculating Triggers
max_Bu = sentiment_data['Bullish'].max()
min_Bu = sentiment_data['Bullish'].min()
mean_Bu = sentiment_data['Bullish'].mean()
mid_Bu = (max_Bu + mean_Bu)/2

max_Be = sentiment_data['Bearish'].max()
min_Be = sentiment_data['Bearish'].min()
mean_Be = sentiment_data['Bearish'].mean()
mid_Be = (max_Be + mean_Be)/2

print ('=========')
print (f"Max Bullish : {max_Bu}")
print (f"Min Bullish : {min_Bu}")
print (f"Avg Bullish : {mean_Bu}")
print (f"Mid Bullish : {mid_Bu}")
print ('=========')
print (f"Max Bearish : {max_Be}")
print (f"Min Bearish : {min_Be}")
print (f"Avg Bearish : {mean_Be}")
print (f"Mid Bearish : {mid_Be}")
print ('=========')

Max Bullish : 0.597531
Min Bullish : 0.203085
Avg Bullish : 0.3488052788461538
Mid Bullish : 0.4731681394230769
Max Bearish : 0.502976
Min Bearish : 0.155556
Avg Bearish : 0.30672322115384615
Mid Bearish : 0.40484961057692304


In [29]:
for i in range(len(sentiment_data)-2):
    
    aux_df = sp500_data.loc[
        sentiment_data['Date'][i]
        :sentiment_data['Date'][i+2]
    ].reset_index()

In [30]:
aux_df.head()

,Date,Close,%Change
0,2019-12-12,3168.57,0.008575
1,2019-12-13,3168.80,0.000073
2,2019-12-16,3191.45,0.007148
3,2019-12-17,3192.52,0.000335
4,2019-12-18,3191.14,-0.000432


In [31]:
aux_df['%Change'][1]

7.258794976916505e-05

In [32]:
i=0
consolidated_dict = {}
consolidated_dict = {'Date':['T+1','T+2','T+3','T+4','Bu/Be/Ne', 'Strength']}

for i in range(len(sentiment_data)-2):

    aux_df = sp500_data.loc[
        sentiment_data['Date'][i]
        :sentiment_data['Date'][i+2]
    ].reset_index()
    
#     Assigning rules for marking the set as neutral.
    if sentiment_data['Bullish'][i] == sentiment_data['Bearish'][i] or max( #if the bullish value is the same as the bearish value
        sentiment_data['Bullish'][i],                                       # OR if the max out of the three is the same as neutral, THEN
        sentiment_data['Bearish'][i],
        sentiment_data['Neutral'][i]
    ) == sentiment_data['Neutral'][i]:
        
        consolidated_dict[str(sentiment_data['Date'][i])] = [               # Fill the value of the dict with the date of that row,
            aux_df['%Change'][1],                                           # and fill the T+1...T+4 columns with the first 4 values of the 
            aux_df['%Change'][2],                                           # %Change column
            aux_df['%Change'][3],
            aux_df['%Change'][4],
            'Ne',
            np.nan
        ]

#    Assigning the rules for marking the bullish set
    elif max(sentiment_data['Bullish'][i], 
             sentiment_data['Bearish'][i], 
             sentiment_data['Neutral'][i]
            ) == sentiment_data['Bullish'][i]:
        
        
        if sentiment_data['Bullish'][i] >= mid_Bu:
            
            consolidated_dict[
                str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Bu',
                'Strong'
            ]
            
            
        elif sentiment_data['Bullish'][i] >= mean_Bu and sentiment_data['Bullish'][i] < mid_Bu:
            
            consolidated_dict[str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Bu',
                'Moderate']
            
        else:
            consolidated_dict[str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Bu',
                'Weak'
            ]
       

#     Setting the rules to mark the set bearish
    elif max(
        sentiment_data['Bullish'][i],
        sentiment_data['Bearish'][i],
        sentiment_data['Neutral'][i]) == sentiment_data['Bearish'][i]:
        
        if sentiment_data['Bearish'][i] >= mid_Be:
            
            consolidated_dict[str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Be',
                'Strong'
            ]
            
        elif sentiment_data['Bearish'][i] >= mean_Be and sentiment_data['Bearish'][i] < mid_Be:
            
            consolidated_dict[str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Be',
                'Moderate'
            ]
            
        else:
            consolidated_dict[str(sentiment_data['Date'][i])] = [
                aux_df['%Change'][1],
                aux_df['%Change'][2],
                aux_df['%Change'][3],
                aux_df['%Change'][4],
                'Be',
                'Weak'
            ]        
    i+=1

aux_df = pd.DataFrame.from_dict(consolidated_dict, orient='index')

aux_df.reset_index(inplace=True)
aux_df.columns = ['Date', 'T+1','T+2','T+3','T+4', 'Bu_Be_Ne', 'Strength']
aux_df.drop([0], inplace=True)
aux_df['Date'] = pd.DatetimeIndex(aux_df['Date'])
aux_df.tail(20)

,Date,T+1,T+2,T+3,T+4,Bu_Be_Ne,Strength
83,2019-08-01,-0.00728274,-0.0297778,0.013017,0.00076689,Bu,Moderate
84,2019-08-08,-0.00661654,-0.0121837,0.0149943,-0.0292928,Be,Strong
85,2019-08-15,0.0144262,0.0121059,-0.00791476,0.00824683,Be,Strong
86,2019-08-22,-0.0259464,0.0109831,-0.00320319,0.00654547,Be,Moderate
87,2019-08-29,0.000642827,-0.00689912,0.0108421,0.0130098,Be,Strong
88,2019-09-05,0.000910618,-9.40004e-05,0.000322317,0.00722967,Be,Moderate
89,2019-09-12,-0.000724356,-0.00313561,0.00258176,0.000342682,Ne,NaN
90,2019-09-19,-0.00489559,-9.69229e-05,-0.00841639,0.00615857,Ne,NaN
91,2019-09-26,-0.00531633,0.00504762,-0.0122584,-0.0179032,Ne,NaN
92,2019-10-03,0.0142169,-0.0044783,-0.0155608,0.00910455,Be,Moderate


## Interpreting the results above:
1.Initialized a for loop the length of the sentiment data set - 2 weeks.

2.Calculated "Ranges" for bearish, bullish, and neutral.

3.Took assigned the auxdf index the length of sentiment data set for each instance i, up to i+2. That gives us two weeks of data.

4.Looked at each value of of i in the sentiment dataframe and compared it to the range of bearish, bullish, and neutral. 

5.Took the date of i in the sentiment dataframe and assigned it to the dictionary KEY, and then took the next 4 instances of returns (i+1...4) and assigned them, as the KEYS. 

6.There are two other KEYS assigned: a Be/Bu/Ne key, and a "strength" key. These are defined by rules in the code. 

7.We then turned that into a dataframe with all information.

In [33]:
# Concatenate sentiment_data & aux_df DataFrames into a single DataFrame
sentiment_data.set_index(sentiment_data['Date'], inplace = True)
aux_df.set_index(aux_df['Date'], inplace=True)
joined_data_cols = pd.concat([sentiment_data, aux_df], axis='columns', join='inner')
joined_data_cols.drop(columns = 'Date',inplace = True)
joined_data_cols.head()

,Bullish,Neutral,Bearish,T+1,T+2,T+3,T+4,Bu_Be_Ne,Strength
Date,,,,,,,,,
2018-01-04,0.597531,0.246914,0.155556,0.0070338,0.00166232,0.0013029,-0.00111221,Bu,Strong
2018-01-11,0.486726,0.262537,0.250737,0.00674963,-0.00352446,0.009415,-0.00161638,Bu,Strong
2018-01-18,0.541133,0.244973,0.213894,0.00438523,0.00806675,0.0021744,-0.000560031,Bu,Strong
2018-01-25,0.454545,0.305195,0.240260,0.0118412,-0.00673194,-0.0108988,0.00048894,Bu,Moderate
2018-02-01,0.447712,0.264706,0.287582,-0.0212085,-0.0409792,0.0174409,-0.0050016,Bu,Moderate


In [34]:
joined_data_cols['Bullish'] = joined_data_cols['Bullish'].astype(float)
joined_data_cols['Neutral'] = joined_data_cols['Neutral'].astype(float)
joined_data_cols['Bearish'] = joined_data_cols['Bearish'].astype(float)
joined_data_cols['T+1'] = joined_data_cols['T+1'].astype(float)
joined_data_cols['T+2'] = joined_data_cols['T+2'].astype(float)
joined_data_cols['T+3'] = joined_data_cols['T+3'].astype(float)
joined_data_cols['T+4'] = joined_data_cols['T+4'].astype(float)
joined_data_cols.dtypes

Bullish     float64
Neutral     float64
Bearish     float64
T+1         float64
T+2         float64
T+3         float64
T+4         float64
Bu_Be_Ne     object
Strength     object
dtype: object

--------------------------------------------------------------------

## Now we are adding a range for each date: This is the DELTA between bearish and bullish values.

In [35]:
joined_data_cols['Diff_in_Opinions'] = joined_data_cols[['Bullish', 'Bearish']].max(axis=1)-joined_data_cols[['Bullish', 'Bearish']].min(axis=1)

In [36]:
joined_data_cols['Diff_in_Opinions'] = joined_data_cols['Diff_in_Opinions'].astype(float)
joined_data_cols

,Bullish,Neutral,Bearish,T+1,T+2,T+3,T+4,Bu_Be_Ne,Strength,Diff_in_Opinions
Date,,,,,,,,,,
2018-01-04,0.597531,0.246914,0.155556,0.007034,0.001662,0.001303,-0.001112,Bu,Strong,0.441975
2018-01-11,0.486726,0.262537,0.250737,0.006750,-0.003524,0.009415,-0.001616,Bu,Strong,0.235989
2018-01-18,0.541133,0.244973,0.213894,0.004385,0.008067,0.002174,-0.000560,Bu,Strong,0.327239
2018-01-25,0.454545,0.305195,0.240260,0.011841,-0.006732,-0.010899,0.000489,Bu,Moderate,0.214285
2018-02-01,0.447712,0.264706,0.287582,-0.021209,-0.040979,0.017441,-0.005002,Bu,Moderate,0.160130
...,...,...,...,...,...,...,...,...,...,...
2019-11-14,0.407229,0.344578,0.248193,0.007695,0.000503,-0.000593,-0.003756,Bu,Moderate,0.159036
2019-11-21,0.342432,0.367246,0.290323,0.002175,0.007507,0.002196,0.004174,Ne,NaN,0.052109
2019-11-29,0.336391,0.360856,0.302752,-0.008631,-0.006638,0.006324,0.001500,Ne,NaN,0.033639


In [37]:
joined_data_cols.isnull().sum()

Bullish              0
Neutral              0
Bearish              0
T+1                  0
T+2                  0
T+3                  0
T+4                  0
Bu_Be_Ne             0
Strength            39
Diff_in_Opinions     0
dtype: int64

In [38]:
joined_data_cols.dropna(inplace = True)

In [39]:
joined_data_cols.count()

Bullish             63
Neutral             63
Bearish             63
T+1                 63
T+2                 63
T+3                 63
T+4                 63
Bu_Be_Ne            63
Strength            63
Diff_in_Opinions    63
dtype: int64

In [40]:
joined_data_cols.head()

,Bullish,Neutral,Bearish,T+1,T+2,T+3,T+4,Bu_Be_Ne,Strength,Diff_in_Opinions
Date,,,,,,,,,,
2018-01-04,0.597531,0.246914,0.155556,0.007034,0.001662,0.001303,-0.001112,Bu,Strong,0.441975
2018-01-11,0.486726,0.262537,0.250737,0.006750,-0.003524,0.009415,-0.001616,Bu,Strong,0.235989
2018-01-18,0.541133,0.244973,0.213894,0.004385,0.008067,0.002174,-0.000560,Bu,Strong,0.327239
2018-01-25,0.454545,0.305195,0.240260,0.011841,-0.006732,-0.010899,0.000489,Bu,Moderate,0.214285
2018-02-01,0.447712,0.264706,0.287582,-0.021209,-0.040979,0.017441,-0.005002,Bu,Moderate,0.160130


In [41]:
max_Sp = joined_data_cols['Diff_in_Opinions'].max()
min_Sp = joined_data_cols['Diff_in_Opinions'].min()
mean_Sp = joined_data_cols['Diff_in_Opinions'].mean()
mid_Sp = (max_Sp + mean_Sp)/2

print ('=========')
print (f"Max Spread : {max_Sp}")
print (f"Min Spread : {min_Sp}")
print (f"Avg Spread : {mean_Sp}")
print (f"Mid Spread : {mid_Sp}")
print ('=========')

Max Spread : 0.441975
Min Spread : 0.008772000000000002
Avg Spread : 0.1449984761904762
Mid Spread : 0.2934867380952381


In [43]:
joined_data_cols.columns

Index(['Bullish', 'Neutral', 'Bearish', 'T+1', 'T+2', 'T+3', 'T+4', 'Bu_Be_Ne',
       'Strength', 'Diff_in_Opinions'],
      dtype='object')

In [45]:
joined_data_cols.rename(columns={
    'Bu_Be_Ne': 'sentiment',
    'T+1':'t1',
    'T+2':'t2',
    'T+3':'t3',
    'T+4':'t4',
    'Diff_in_Opinions':'sent_spread'
}, inplace=True)

In [ ]:
d = {}
joined_data_cols.reset_index(inplace = True)

for row in joined_data_cols.itertuples():
    
    if row.sentiment is 'Bu':
        if row.t1 > 0:
            if row.sent_spread >= mid_Sp:
                d[row.date] = ['h', 'y']
            elif row.sent_spread < mean_Sp:
                d[row.date] = ['l', 'y']
            else:
                d[row.date] = ['m', 'y']
                
        else:
            if row.sent_spread >= mid_Sp:
                d[row.date] = ['h', 'n']
            elif row.sent_spread < mean_Sp:
                d[row.date] = ['l', 'n']
            else:
                d[row.date] = ['m', 'n']
                
                
    if row.sentiment is 'Be':
        if row.t1 < 0:
            if row.sent_spread >= mid_Sp:
                d[row.date] = ['h', 'y']
            elif row.sent_spread < mean_Sp:
                d[row.date] = ['l', 'y']
            else:
                d[row.date] = ['m', 'y']
                
        else:
            if row.sent_spread >= mid_Sp:
                d[row.date] = ['h', 'n']
            elif row.sent_spread < mean_Sp:
                d[row.date] = ['l', 'n']
            else:
                d[row.date] = ['m', 'n']

In [21]:
i=0
spread_dict = {}
joined_data_cols.reset_index(inplace = True)
for i in range(len(joined_data_cols)):
#     if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+1'][i] > 0:
#         if joined_data_cols['Diff_in_Opinions'][i] >= mid_Sp:
#             spread_dict[joined_data_cols['Date'][i]] = ['High', 'Y']
#         elif joined_data_cols['Diff_in_Opinions'][i] >= mean_Sp and joined_data_cols['Diff_in_Opinions'][i] < mid_Sp:
#             spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'Y']
#         else:
#             spread_dict[joined_data_cols['Date'][i]] = ['Low','Y']
            
            
#     elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+1'][i] <= 0:
#         if joined_data_cols['Diff_in_Opinions'][i] >= mid_Sp:
#             spread_dict[joined_data_cols['Date'][i]] = ['High', 'N']
#         elif joined_data_cols['Diff_in_Opinions'][i] >= mean_Sp and joined_data_cols['Diff_in_Opinions'][i] < mid_Sp:
#             spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'N']
#         else:
#             spread_dict[joined_data_cols['Date'][i]] = ['Low','N']
            
            
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+1'][i] < 0:
        if joined_data_cols['Diff_in_Opinions'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'Y']
        elif joined_data_cols['Diff_in_Opinions'][i] >= mean_Sp and joined_data_cols['Diff_in_Opinions'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'Y']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','Y']
            
            
    elif joined_data_cols['Diff_in_Opinions'][i] == 'Be' and joined_data_cols['T+1'][i] >= 0:
        if joined_data_cols['Diff_in_Opinions'][i] >= mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['High', 'N']
        elif joined_data_cols['Diff_in_Opinions'][i] >= mean_Sp and joined_data_cols['Diff_in_Opinions'][i] < mid_Sp:
            spread_dict[joined_data_cols['Date'][i]] = ['Mid', 'N']
        else:
            spread_dict[joined_data_cols['Date'][i]] = ['Low','N']
    

In [21]:
   #ADD A FOR LOOP AND VALIDATE AFTER YOU CAN VISUALIZE THE CELL ABOVE
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+2'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+2'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+2'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+2'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+3'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+3'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+3'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+3'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    
    if joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+4'][i] > 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Bu' and joined_data_cols['T+4'][i] <= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+4'][i] < 0:
        spread_dict[joined_data_cols['Date'][i]].append('Y')
    elif joined_data_cols['Bu_Be_Ne'][i] == 'Be' and joined_data_cols['T+4'][i] >= 0:
        spread_dict[joined_data_cols['Date'][i]].append('N')
        
    i+=1
    
aux_df = pd.DataFrame.from_dict(spread_dict, orient='index')

aux_df.reset_index(inplace=True)
aux_df.columns = ['Date', 'Spread', 'MatchT+1', 'MatchT+2', 'MatchT+3', 'MatchT+4']
aux_df['Date'] = pd.DatetimeIndex(aux_df['Date'])

aux_df.set_index(aux_df['Date'], inplace=True)
aux_df.drop(columns = 'Date', inplace = True)
aux_df.tail()

KeyError: Timestamp('2018-06-28 00:00:00')

In [ ]:
#Concatenate joined_data_cols & aux_df
joined_data_cols.set_index(joined_data_cols['Date'], inplace=True)
joined_data_cols = pd.concat([joined_data_cols, aux_df], axis='columns', join='inner')
joined_data_cols.drop(columns = 'Date',inplace = True)
joined_data_cols.tail()
joined_data_cols.to_csv("Check_data.csv", encoding='utf-8')#, index=False)

In [ ]:
joined_data_cols.groupby(['Bu_Be_Ne','Strength','Spread','MatchT+2']).count()

In [ ]:
joined_data_cols.groupby(['Bu_Be_Ne','Strength','Spread']).count()